<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.3.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Shuffling and writing examples to /Users/giacomomiolo/tensorflow_datasets/horses_or_humans/3.0.0.incompleteDB68RE/horses_or_humans-train.tfrecord


Shuffling and writing examples to /Users/giacomomiolo/tensorflow_datasets/horses_or_humans/3.0.0.incompleteDB68RE/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /Users/giacomomiolo/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [ ]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

In [7]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = tf.keras.models.Sequential([
                                        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
                                        tf.keras.layers.MaxPooling2D(2, 2),
                                        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                                        tf.keras.layers.MaxPooling2D(2, 2),
                                        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                                        tf.keras.layers.MaxPooling2D(2, 2),
                                        tf.keras.layers.Flatten(),
                                        tf.keras.layers.Dense(dense_units, activation='relu'),
                                        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [ ]:
# !rm -rf logs

In [8]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
 1/26 [>.............................] - ETA: 0s - loss: 0.6891 - accuracy: 0.6250WARNING:tensorflow:From /Users/giacomomiolo/opt/anaconda3/envs/py386_tf23/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


26/26 [==============================] - 11s 442ms/step - loss: 0.6721 - accuracy: 0.5925 - val_loss: 0.6570 - val_accuracy: 0.5951
Epoch 2/10
26/26 [==============================] - 12s 464ms/step - loss: 0.6222 - accuracy: 0.6764 - val_loss: 0.6174 - val_accuracy: 0.6439
Epoch 3/10
26/26 [==============================] - 13s 506ms/step - loss: 0.5625 - accuracy: 0.7543 - val_loss: 0.6967 - val_accuracy: 0.5268
Epoch 4/10
26/26 [==============================] - 12s 471ms/step - loss: 0.5120 - accuracy: 0.7737 - val_loss: 0.5503 - val_accuracy: 0.6927
Epoch 5/10
26/26 [==============================] - 12s 455ms/step - loss: 0.4602 - accuracy: 0.8090 - val_loss: 0.4520 - val_accuracy: 0.8537
Epoch 6/10
26/26 [==============================] - 11s 437ms/step - loss: 0.3976 - accuracy: 0.8455 - val_loss: 0.3801 - val_accuracy: 0.8878
Epoch 7/10
26/26 [==============================] - 11s 412ms/step - loss: 0.3255 - accuracy: 0.8808 - val_loss: 0.3068 - val_accuracy: 0.8634
Epoch 8/10

In [9]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.66.h5
26/26 - 11s - loss: 0.6758 - accuracy: 0.5535 - val_loss: 0.6573 - val_accuracy: 0.7854
Epoch 2/5

Epoch 00002: saving model to weights.02-0.60.h5
26/26 - 13s - loss: 0.6309 - accuracy: 0.7141 - val_loss: 0.6025 - val_accuracy: 0.8000
Epoch 3/5

Epoch 00003: saving model to weights.03-0.64.h5
26/26 - 12s - loss: 0.5745 - accuracy: 0.7774 - val_loss: 0.6442 - val_accuracy: 0.5024
Epoch 4/5

Epoch 00004: saving model to weights.04-0.51.h5
26/26 - 11s - loss: 0.5180 - accuracy: 0.7774 - val_loss: 0.5065 - val_accuracy: 0.7805
Epoch 5/5

Epoch 00005: saving model to weights.05-0.45.h5
26/26 - 11s - loss: 0.4449 - accuracy: 0.8200 - val_loss: 0.4528 - val_accuracy: 0.8390


In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 14s - loss: 0.6763 - accuracy: 0.5487 - val_loss: 0.6656 - val_accuracy: 0.4927


In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 10s - loss: 0.6749 - accuracy: 0.5693 - val_loss: 0.6683 - val_accuracy: 0.5659
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 13s - loss: 0.6415 - accuracy: 0.6667 - val_loss: 0.6280 - val_accuracy: 0.6146


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 12s - loss: 0.6683 - accuracy: 0.5864 - val_loss: 0.7414 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 12s - loss: 0.6178 - accuracy: 0.6922 - val_loss: 0.5775 - val_accuracy: 0.7561
Epoch 3/50
26/26 - 10s - loss: 0.5495 - accuracy: 0.7676 - val_loss: 0.5199 - val_accuracy: 0.7902
Epoch 4/50
26/26 - 10s - loss: 0.5301 - accuracy: 0.7762 - val_loss: 0.4355 - val_accuracy: 0.8439
Epoch 5/50
26/26 - 10s - loss: 0.4616 - accuracy: 0.8151 - val_loss: 0.3891 - val_accuracy: 0.8732
Epoch 6/50
26/26 - 10s - loss: 0.3781 - accuracy: 0.8650 - val_loss: 0.3205 - val_accuracy: 0.8634
Epoch 7/50
26/26 - 11s - loss: 0.3098 - accuracy: 0.8978 - val_loss: 0.3461 - val_accuracy: 0.8732
Epoch 8/50
26/26 - 13s - loss: 0.2634 - accuracy: 0.9197 - val_loss: 0.2176 - val_accuracy: 0.9463
Epoch 9/50
26/26 - 10s - loss: 0.2030 - accuracy: 0.9428 - val_loss: 0.1824 - val_accuracy: 0.9366
Epoch 10/50
26/26 - 10s - loss: 0.1640 - accuracy: 0.9550 - val_loss: 0.1254 - val_accuracy: 0.9756
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 11s 415ms/step - loss: 0.6721 - accuracy: 0.5888 - val_loss: 0.6671 - val_accuracy: 0.5220
Epoch 2/5
26/26 [==============================] - 11s 426ms/step - loss: 0.6361 - accuracy: 0.6715 - val_loss: 0.6265 - val_accuracy: 0.6244
Epoch 3/5
26/26 [==============================] - 11s 422ms/step - loss: 0.5814 - accuracy: 0.7202 - val_loss: 0.5397 - val_accuracy: 0.8049
Epoch 4/5
26/26 [==============================] - 10s 389ms/step - loss: 0.5122 - accuracy: 0.7835 - val_loss: 0.4827 - val_accuracy: 0.8488
Epoch 5/5
26/26 [==============================] - 13s 490ms/step - loss: 0.4697 - accuracy: 0.7968 - val_loss: 0.4313 - val_accuracy: 0.8537


In [15]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.588808,0.672073,0.521951,0.667128
1,1,0.671533,0.636129,0.624390,0.626477
2,2,0.720195,0.581357,0.804878,0.539652
3,3,0.783455,0.512243,0.848781,0.482679
4,4,0.796837,0.469744,0.853659,0.431287


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 12s 443ms/step - loss: 0.6724 - accuracy: 0.5596 - val_loss: 0.6777 - val_accuracy: 0.4683

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 11s 417ms/step - loss: 0.6539 - accuracy: 0.5937 - val_loss: 0.6733 - val_accuracy: 0.4585

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 11s 414ms/step - loss: 0.6427 - accuracy: 0.5852 - val_loss: 0.6601 - val_accuracy: 0.5659

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 11s 408ms/step - loss: 0.6376 - accuracy: 0.6521 - val_loss: 0.6566 - val_accuracy: 0.5756

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 11s 426ms/step - loss: 0.6

In [17]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 12s 448ms/step - loss: 0.6764 - accuracy: 0.5645 - val_loss: 0.6657 - val_accuracy: 0.6146
Epoch 2/50
26/26 [==============================] - 11s 404ms/step - loss: 0.6479 - accuracy: 0.6776 - val_loss: 0.6623 - val_accuracy: 0.5268
Epoch 3/50
26/26 [==============================] - 11s 413ms/step - loss: 0.6076 - accuracy: 0.6995 - val_loss: 0.5727 - val_accuracy: 0.8488
Epoch 4/50
26/26 [==============================] - 11s 417ms/step - loss: 0.5605 - accuracy: 0.7311 - val_loss: 0.5182 - val_accuracy: 0.8488
Epoch 5/50
26/26 [==============================] - 11s 439ms/step - loss: 0.4921 - accuracy: 0.7932 - val_loss: 0.4756 - val_accuracy: 0.8098
Epoch 6/50
26/26 [==============================] - 11s 415ms/step - loss: 0.4553 - accuracy: 0.8175 - val_loss: 0.4384 - val_accuracy: 0.7463
Epoch 7/50
26/26 [==============================] - 11s 440ms/step - loss: 0.3984 - accuracy: 0.8431 - val_loss: 0.3189 - val_accuracy: 0.9366

In [19]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 62193), started 0:11:10 ago. (Use '!kill 62193' to kill it.)